In [4]:
import os
import random
import cv2
import datetime
import numpy as np
import shutil
class run_img_maker:
    data_folder = 'DATA/run_certification'
    save_path = 'result'
    kakao_img = r'C:\Users\DW\Documents\카카오톡 받은 파일'
    
    def __init__(self, greed_size=[0,0], full_img_size = 3240, a_img_size = 1080, target_month = [], lastest = False) -> None:
        self.full_img_size = full_img_size
        self.a_img_size = a_img_size
        self.target_month = target_month
        self.lastest = lastest
        self.ETL_img()
        self.img_list = self.get_target_img_list(self.target_month, self.lastest)
        self.greed_size = self.get_greed_size(greed_size)
        self.make_img_num = self.get_img_num()

    def ETL_img(self):
        k_list = os.listdir(self.kakao_img)
        for i in k_list:
            i_sp = i.split('_')
            date_dt = i_sp[1]
            month = date_dt[:6]
            origin = os.path.join(self.kakao_img,i)
            new = os.path.join(self.data_folder,month,i)
            if not os.path.exists(os.path.join(self.data_folder,month)):
                os.mkdir(os.path.join(self.data_folder,month))
            shutil.copy(origin,new)
            os.remove(origin)

    def get_date(self,file_name:str):
        f_sp = file_name.split('_')
        date_dt = f_sp[1]
        #month = date_dt
        return date_dt

    def get_target_img_list(self,target_month, lastest):
        img_path_list = []
        lastest_list = []
        if target_month == []:
            target_month = os.listdir(self.data_folder)
        for m_folder in target_month:
            img_list = os.listdir(os.path.join(self.data_folder,m_folder))
            for a_img in img_list:
                a_img_path = os.path.join(self.data_folder,m_folder,a_img)
                img_path_list.append(a_img_path)
                lastest_list.append(self.get_date(a_img))
        random.shuffle(img_path_list)

        dumi = []
        if lastest:
            last_set = list(set(lastest_list))
            last_set.sort(reverse=True)
            last_set = last_set[:2]
            for i in img_path_list:
                for y_date in last_set:
                    if y_date in i.split('_')[2]:
                        dumi.append(i)
                        print(y_date,i)
            img_path_list = dumi
        print('__img__len = ',len(img_path_list))
        return img_path_list
    
    def get_greed_size(self,greed_size):
        img_len = len(self.img_list)
        if greed_size == [0,0]:
            # auto
            a_greed = 1
            while 1:
                if a_greed * a_greed < img_len:
                    a_greed += 1
                else:
                    greed_size = [a_greed,a_greed]
                    break
        return greed_size
            
    def get_img_num(self):
        a_need_img_num = self.greed_size[0] * self.greed_size[1]
        total_img_num = len(self.img_list)
        print(total_img_num,'total_img_num')
        make_img = total_img_num//a_need_img_num
        print(make_img,'make_img')
        if make_img == 0:
            duplicate_num = self.greed_size[0] * self.greed_size[1] - total_img_num
            print(duplicate_num,'duplicate_num')
            duplicate_list = random.sample(self.img_list,duplicate_num)
            self.img_list = self.img_list + duplicate_list
            print(len(self.img_list),'len(self.img_list)')
            make_img = 1
        return make_img

    def img_squar_paddingNresize(self,image,size):
        # 이미지의 가로, 세로 크기 구하기
        height, width = image.shape[:2]
        #size = max(height,width)

        # 정사각형의 크기로 리사이즈하기
        if height > width:
            diff = height - width
            pad = diff // 2
            resized_image = cv2.copyMakeBorder(image, 0, 0, pad, pad, cv2.BORDER_CONSTANT)
        else:
            diff = width - height
            pad = diff // 2
            resized_image = cv2.copyMakeBorder(image, pad, pad, 0, 0, cv2.BORDER_CONSTANT)

        # 지정한 크기로 이미지 리사이즈하기
        resized_image = cv2.resize(resized_image, (size, size))

        return resized_image
    
    def make_img(self):
        for i in range(self.make_img_num):
            today = datetime.date.today()
            today = today.strftime("%Y-%m-%d")
            tag = f'{self.greed_size[0]}x{self.greed_size[1]}'
            result_file_name = os.path.join(self.save_path,f'{today}_{tag}_{i}.jpg')
            make_a_img = self._make_a_img()
            make_a_img = cv2.resize(make_a_img, (self.full_img_size, self.full_img_size))
            cv2.imwrite(result_file_name,make_a_img)

    def _make_a_img(self):
        print(self.greed_size,'self.greed_size')
        canvas = np.zeros((self.greed_size[1]*self.a_img_size, self.greed_size[0]*self.a_img_size, 3), dtype=np.uint8)
        for y in range(self.greed_size[0]):
            for x in range(self.greed_size[1]):
                print(x,y)
                target_img_path = self.img_list.pop(0)
                target_cv_img = cv2.imread(target_img_path)
                padding_resized_img = self.img_squar_paddingNresize(target_cv_img,self.a_img_size)
                try:
                    canvas[x*self.a_img_size:(x+1)*self.a_img_size, y*self.a_img_size:(y+1)*self.a_img_size] = padding_resized_img
                except:
                    pass
        return canvas
    
greed_size=[3,4]
target_month = ['202308']
lastest = True      # 최신본만 가지고 제작
# rim = run_img_maker(target_month = target_month, lastest = lastest)  # 최신본만 가지고 제작
#rim = run_img_maker(target_month = target_month)
#rim = run_img_maker()
rim = run_img_maker(greed_size = [5,5],target_month=target_month) # 이번달꺼 5*5로 뽑기
rim.make_img()
    

__img__len =  163
163 total_img_num
6 make_img
[5, 5] self.greed_size
0 0
1 0
2 0
3 0
4 0
0 1
1 1
2 1
3 1
4 1
0 2
1 2
2 2
3 2
4 2
0 3
1 3
2 3
3 3
4 3
0 4
1 4
2 4
3 4
4 4
[5, 5] self.greed_size
0 0
1 0
2 0
3 0
4 0
0 1
1 1
2 1
3 1
4 1
0 2
1 2
2 2
3 2
4 2
0 3
1 3
2 3
3 3
4 3
0 4
1 4
2 4
3 4
4 4
[5, 5] self.greed_size
0 0
1 0
2 0
3 0
4 0
0 1
1 1
2 1
3 1
4 1
0 2
1 2
2 2
3 2
4 2
0 3
1 3
2 3
3 3
4 3
0 4
1 4
2 4
3 4
4 4
[5, 5] self.greed_size
0 0
1 0
2 0
3 0
4 0
0 1
1 1
2 1
3 1
4 1
0 2
1 2
2 2
3 2
4 2
0 3
1 3
2 3
3 3
4 3
0 4
1 4
2 4
3 4
4 4
[5, 5] self.greed_size
0 0
1 0
2 0
3 0
4 0
0 1
1 1
2 1
3 1
4 1
0 2
1 2
2 2
3 2
4 2
0 3
1 3
2 3
3 3
4 3
0 4
1 4
2 4
3 4
4 4
[5, 5] self.greed_size
0 0
1 0
2 0
3 0
4 0
0 1
1 1
2 1
3 1
4 1
0 2
1 2
2 2
3 2
4 2
0 3
1 3
2 3
3 3
4 3
0 4
1 4
2 4
3 4
4 4
